In [1]:
import spotipy
import spotipy.util as util
import json
import pandas as pd

In [2]:
scope = 'playlist-read-private playlist-read-collaborative user-library-read user-read-recently-played user-top-read'
redirect_uri = "http://localhost/"
username = ""
client_id = ""
client_secret = ""

In [3]:
data = pd.read_csv("2018-2020topchart.csv")
print(data.head())

   Rank          Artist                             Song    Streams    Date
0     1     Post Malone                         rockstar  1,502,394  1/1/18
1     2          G-Eazy                         No Limit  1,027,039  1/1/18
2     3        Lil Pump                       Gucci Gang    930,620  1/1/18
3     4         Cardi B  Bartier Cardi (feat. 21 Savage)    877,478  1/1/18
4     5  Camila Cabello                           Havana    860,232  1/1/18


In [4]:
token = util.prompt_for_user_token(username,scope,client_id,client_secret,redirect_uri)
sp = spotipy.Spotify(auth=token)

jsonPrettify = lambda x: json.dumps(x, indent=2)

In [23]:
from string import punctuation
import time

def findSongId(trackName, artistName):
    trackName = trackName.split("(feat.")[0].strip()
    trackName = trackName.replace("'","").replace("!","")
    offset = 0
    while offset < 500:
        try:
            global sp
            results = sp.search(q="track:" + trackName, type="track", limit=10, offset=offset)
        except spotipy.client.SpotifyException:
            token = util.prompt_for_user_token(username,scope,client_id,client_secret,redirect_uri)
            sp = spotipy.Spotify(auth=token)
            results = sp.search(q="track:" + trackName, type="track", limit=10, offset=offset)
        items = results['tracks']['items']
        
        trackId = ""
        for track in items:
            trackId = track["id"]
            for artist in track["artists"]:
                if artist["name"] == artistName and track["popularity"] > 0:
    #                 print(jsonPrettify(track))
                    return trackId
        offset += 10
        time.sleep(0.5)
    return ""

In [25]:
id = findSongId("rockstar", "Post Malone")
print(id)
print(sp.audio_features(id))

0e7ipj03S05BNilyu5bRzt
[{'danceability': 0.585, 'energy': 0.52, 'key': 5, 'loudness': -6.136, 'mode': 0, 'speechiness': 0.0712, 'acousticness': 0.124, 'instrumentalness': 7.01e-05, 'liveness': 0.131, 'valence': 0.129, 'tempo': 159.801, 'type': 'audio_features', 'id': '0e7ipj03S05BNilyu5bRzt', 'uri': 'spotify:track:0e7ipj03S05BNilyu5bRzt', 'track_href': 'https://api.spotify.com/v1/tracks/0e7ipj03S05BNilyu5bRzt', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/0e7ipj03S05BNilyu5bRzt', 'duration_ms': 218147, 'time_signature': 4}]


In [62]:
from time import time
from time import sleep
from random import randint

songValues = {}
allRows = []

In [72]:
df = pd.DataFrame([], columns=["Rank","Artist","Title","Streams","Date",
                                           "Id","Duration","Time Signature", "Tempo",
                                           "Key", "Mode"," Valence", "Danceability",
                                           "Energy", "Acousticness", "Instrumentalness"])
with open("topChartWithFeatures.csv","a") as f:
    df.to_csv(f,header=True,index=False)

start = time()
dayTime = time()
prevDate = ""
for i in range(len(data)):
    row = data.iloc[i]
    allRows = []
    artistSong = row["Artist"] + "/:/" + row["Song"]
    currDate = row["Date"]
    if currDate != prevDate:
        print("Day: {}, Row: {} Time: {}".format(currDate, i, time() - dayTime))
        dayTime = time()
        prevDate = currDate
        
    if artistSong not in songValues:
        song = row["Song"]
        artist = row["Artist"]
        sleep(randint(0,2))
        songId = findSongId(song, artist)
        if songId == "":
            print("Error with: {} by {}".format(song, artist))
            rowdata = [row["Rank"], artist, song, row["Streams"], row["Date"],
                   songId, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
        else:
            features = sp.audio_features(songId)[0]
            rowdata = [row["Rank"], artist, song, row["Streams"], row["Date"],
                       songId, features["duration_ms"], features["time_signature"],
                       features["tempo"], features["key"], features["mode"], features["valence"],
                       features["danceability"], features["energy"], features["acousticness"], 
                       features["instrumentalness"]]
        
        songValues[artistSong] = rowdata
    else:
        rowdata = songValues[artistSong]
    allRows.append(rowdata)
    
    if i % 100 == 0:
        df = pd.DataFrame(allRows, columns=["Rank","Artist","Title","Streams","Date",
                                           "Id","Duration","Time Signature", "Tempo",
                                           "Key", "Mode"," Valence", "Danceability",
                                           "Energy", "Acousticness", "Instrumentalness"])
        with open("topChartWithFeatures.csv", "a") as f:
            df.to_csv(f, header=False, index=False)

print("Time to complete:{}".format(time() - start))

Day: 1/1/18, Row: 0
Day: 1/2/18, Row: 200
Day: 1/3/18, Row: 400
Day: 1/4/18, Row: 600
Day: 1/5/18, Row: 800
Day: 1/6/18, Row: 1000
Day: 1/7/18, Row: 1200
Day: 1/8/18, Row: 1400
Day: 1/9/18, Row: 1600
Day: 1/10/18, Row: 1800
Day: 1/11/18, Row: 2000
My My My!
Error with: My My My! by Troye Sivan
Day: 1/12/18, Row: 2200
In the Dark
Error with: In the Dark by Camila Cabello
Day: 1/13/18, Row: 2400
Day: 1/14/18, Row: 2600
Day: 1/15/18, Row: 2800
La Modelo
Error with: La Modelo by Lean Trap
Day: 1/16/18, Row: 3000
Dreams
Error with: Dreams by The Cranberries
Day: 1/17/18, Row: 3200
Do Right
Error with: Do Right by Glades
Day: 1/18/18, Row: 3400
Day: 1/19/18, Row: 3600
Church
Error with: Church by Fall Out Boy
Alone
Error with: Alone by Russ
Day: 1/20/18, Row: 3800
Day: 1/21/18, Row: 4000
Day: 1/22/18, Row: 4200
Champion
Error with: Champion by Fall Out Boy
Day: 1/23/18, Row: 4400
Day: 1/24/18, Row: 4600
Day: 1/25/18, Row: 4800
Day: 1/26/18, Row: 5000
Gang Gang
Error with: Gang Gang by Migos


SpotifyException: http status: 401, code:-1 - https://api.spotify.com/v1/search?q=track%3AGone&limit=10&offset=0&type=track:
 The access token expired